In [46]:

train_data=pd.read_csv('C:/Users/SRIKANT NAYAK/Documents/code/bloom_texonomy/data2.csv',encoding= 'unicode_escape')

new_data=pd.read_csv('C:/Users/SRIKANT NAYAK/Documents/code/bloom_texonomy/upsc.csv',encoding= 'unicode_escape')



def Question_classification(training_data,new_data):
 
   #...........................................import the libreries....................................................
    
    import pandas as pd
    import numpy as np
    import re
    import nltk
    import matplotlib.pyplot as plt
    import re
    import numpy as np
    import matplotlib.pyplot as plt
    from nltk.corpus import stopwords
    from nltk.stem import PorterStemmer
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.pipeline import Pipeline
    from sklearn.metrics import classification_report, confusion_matrix,accuracy_score
    from sklearn.model_selection import train_test_split
    from sklearn.feature_selection import SelectKBest, chi2
    from sqlite3 import Error
    from sklearn.ensemble import RandomForestClassifier
    import sqlite3
    import pickle
    
    #........................................concartinate the training data and the new data.................................
    
    wpt = nltk.WordPunctTokenizer()
    stop_words = nltk.corpus.stopwords.words('english')
    
    df=pd.DataFrame(training_data)
    inputt=df.iloc[0:,0:-1]
    labels=df.Label
    
    new_data=(pd.DataFrame(new_data))

    frame = [inputt,new_data]
    
    total_training_data=pd.concat(frame, axis=0, join='outer', ignore_index=False, keys=None,levels=None, names=None, verify_integrity=False, copy=True)

    
    #...........................................preprocessing the data.........................................................
    
    

    def normalize_document(doc):
        # lower case and remove special characters\whitespaces
        doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
        doc = doc.lower()
        doc = doc.strip()
        # tokenize document
        tokens = wpt.tokenize(doc)
        # filter stopwords out of document
        filtered_tokens = [token for token in tokens if token not in stop_words]
        # re-create document from filtered tokens
        doc = ' '.join(filtered_tokens)
        return doc

    normalize_corpus = np.vectorize(normalize_document)
    norm_total_training_data = normalize_corpus(total_training_data)
    
   
   #......................................convert into vector form................................................... 
    
    def vectors(norm_corpus):
        from sklearn.feature_extraction.text import CountVectorizer
        cv = CountVectorizer(min_df=0., max_df=1.)
        cv_matrix = cv.fit_transform(norm_corpus.ravel())
        cv_matrix = cv_matrix.toarray()
        # get all unique words in the corpus
        vocab = cv.get_feature_names()
        # show document feature vectors
        pd.DataFrame(cv_matrix, columns=vocab)
        # you can set the n-gram range to 1,2 to get unigrams as well as bigrams
        bv = CountVectorizer(ngram_range=(2,2))
        bv_matrix = bv.fit_transform(norm_corpus.ravel())

        bv_matrix = bv_matrix.toarray()
        vocab = bv.get_feature_names()
        pd.DataFrame(bv_matrix, columns=vocab)
        from sklearn.feature_extraction.text import TfidfVectorizer

        tv = TfidfVectorizer(min_df=0., max_df=1., use_idf=True)
        tv_matrix = tv.fit_transform(norm_corpus.ravel())
        tv_matrix = tv_matrix.toarray()

        vocab = tv.get_feature_names()
        pd.DataFrame(np.round(tv_matrix, 2), columns=vocab)
        from sklearn.metrics.pairwise import cosine_similarity

        similarity_matrix = cosine_similarity(tv_matrix)
        similarity_df = pd.DataFrame(similarity_matrix)
        return similarity_df
    
    vector_norm_total_training_data = vectors(norm_total_training_data)
    
    X=vector_norm_total_training_data[0:len(data)]
    
    new_question=vector_norm_total_training_data[len(data):]
    

   # ................................................training the model.....................................................

    #X = df['cleaned']
    Y = df['Label']

    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)
    # instead of doing these steps one at a time, we can use a pipeline to complete them all at once
    from sklearn.svm import LinearSVC
    def support_vector_machine(X_train, y, X_predict):
        lin_clf = LinearSVC()
        lin_clf.fit(X_train, y)
        prediction = lin_clf.predict(X_predict)
        return prediction

    y_pred=support_vector_machine(X_train,y_train,X_test)
    #xtest=pca.transform(x_test)
    from sklearn.metrics import classification_report,accuracy_score
    #y_pred=cls.predict(vectorization.transform(xtest))
    print(accuracy_score(y_test,y_pred))
    print(classification_report(y_test,y_pred))
    class_pred=support_vector_machine(X_train,y_train,new_question)
    
    return class_pred

new_question_class_pred=Question_classification(training_data,new_data)







0.78
               precision    recall  f1-score   support

      Analyze       0.87      0.87      0.87        23
  Application       0.89      0.77      0.83        31
Comprehension       0.86      0.78      0.82        32
   evaluation       0.59      0.64      0.62        25
    knowledge       0.74      0.88      0.80        16
     synthesi       0.72      0.78      0.75        23

     accuracy                           0.78       150
    macro avg       0.78      0.79      0.78       150
 weighted avg       0.79      0.78      0.78       150



In [49]:
(class_pred)

array(['Application', 'Analyze', 'knowledge', 'evaluation', 'synthesi',
       'knowledge', 'evaluation', 'knowledge', 'knowledge', 'knowledge',
       'evaluation', 'knowledge', 'Analyze', 'Application', 'evaluation',
       'Analyze', 'Analyze', 'knowledge', 'knowledge', 'Analyze'],
      dtype=object)

In [59]:
label=pd.DataFrame(class_pred,columns=['Label'])
frame = [upsc,label]
final_test_data=pd.concat(frame, axis=1, join='outer', ignore_index=False, keys=None,levels=None, names=None, verify_integrity=False, copy=True)


In [57]:
final_test_data

,Question,Label
0,The 1857 Uprising was the culmination of the r...,Application
1,Examine the linkages between the ninteenth cen...,Analyze
2,Assess the impact of global warming on the cor...,knowledge
3,Discuss the causes of depletion of mangroves a...,evaluation
4,Can the strategy of regional resource-based ma...,synthesi
5,Discuss the factors for localisation of agro-b...,knowledge
6,What makes the Indian society unique in sustai...,evaluation
7,"""Empowering women is the key to control popula...",knowledge
8,What are the challenges to our cultural practi...,knowledge
9,Many voices had strengthened and enriched the ...,knowledge


In [60]:
#save the output in csv file

final_test_data.to_csv("upsc_classification.csv")